# Scenario Analysis

This notebook demonstrates how to explore scenario outputs via the Aurum API. Configure the environment variables `AURUM_API_BASE_URL` and `AURUM_TENANT_ID` (or edit the cells below) before running.

In [ ]:
import os
import requests
import pandas as pd

BASE_URL = os.getenv('AURUM_API_BASE_URL', 'http://localhost:8095')
TENANT = os.getenv('AURUM_TENANT_ID', '00000000-0000-0000-0000-000000000001')
session = requests.Session()
session.headers.update({'X-Aurum-Tenant': TENANT})
print(f'Using API {BASE_URL} for tenant {TENANT}')


In [ ]:
response = session.get(f'{BASE_URL}/v1/scenarios', params={'limit': 5})
response.raise_for_status()
scenarios = response.json()['data']
pd.DataFrame(scenarios)


In [ ]:
# Replace with a valid scenario_id if needed
scenario_id = scenarios[0]['scenario_id'] if scenarios else None
if scenario_id is None:
    raise RuntimeError('No scenarios available to inspect.')

outputs_resp = session.get(
    f'{BASE_URL}/v1/scenarios/{scenario_id}/outputs',
    params={'limit': 100}
)
outputs_resp.raise_for_status()
outputs = outputs_resp.json()['data']
df = pd.DataFrame(outputs)
df[['run_id', 'curve_key', 'tenor_label', 'metric', 'value', 'asof_date']].head()


In [ ]:
pivot = df.pivot_table(
    index='tenor_label',
    columns='metric',
    values='value'
)
pivot
